# Model Serving Function

In [1]:
import mlrun
import os

In [2]:
import numpy as np
from cloudpickle import load

class LGBMModel(mlrun.serving.V2ModelServer):
    
    def load(self):
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body):
        try:
            feats = np.asarray(body['inputs'])
            result = self.model.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

In [3]:
# nuclio: end-code

## Deploy and Test The Function

In [4]:
models_path = mlrun.get_sample_path('models/lightgbm/SampleModel.pkl')

In [5]:
fn = mlrun.code_to_function('lightgbm-serving',
                            description="LightGBM Serving",
                            categories=['serving', 'ml'],
                            labels={'author': 'edmondg', 'framework': 'lightgbm'},
                            code_output='.',
                            image='mlrun/mlrun',
                            kind='serving')
fn.spec.build.commands = ['pip install lightgbm']
fn.spec.default_class = 'LGBMModel'

In [6]:
fn.add_model('nyc-taxi-server', model_path=models_path)

In [7]:
# deploy the function
fn.apply(mlrun.platforms.auto_mount())
address = fn.deploy()

> 2021-11-16 13:29:40,426 [info] Starting remote function deploy
2021-11-16 13:29:40  (info) Deploying function
2021-11-16 13:29:40  (info) Building
2021-11-16 13:29:40  (info) Staging files and preparing base images
2021-11-16 13:29:40  (info) Building processor image
2021-11-16 13:29:51  (info) Build complete
2021-11-16 13:30:00  (info) Function deploy complete
> 2021-11-16 13:30:01,208 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-default-lightgbm-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['default-lightgbm-serving-default.default-tenant.app.app-lab-eks.iguazio-cd2.com/']}


In [8]:
# test the function
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 3, 5.1, 3.5, 1.4, 0.2, 5.1, 3.5, 1.4, 0.2, 5.1, 3.5, 1.4, 0.2]]}'''
fn.invoke('/v2/models/nyc-taxi-server/predict', my_data)

> 2021-11-16 13:30:01,308 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-default-lightgbm-serving.default-tenant.svc.cluster.local:8080/v2/models/nyc-taxi-server/predict'}


{'id': 'e22543f1-15dc-4563-a5d9-c5c6aed9f716',
 'model_name': 'nyc-taxi-server',
 'outputs': [25.374309065093435]}